In [1]:
from pynq import Overlay
ps_gpio_design = Overlay("./ps_gpio_kv260.bit")

/usr/local/share/pynq-venv/lib/python3.8/site-packages/pynq/pl_server/xrt_device.py:59: UserWarning: xbutil failed to run - unable to determine XRT version
  warnings.warn("xbutil failed to run - unable to determine XRT version")


In [2]:
from pynq import GPIO
from time import sleep

In [14]:


Floor_disp_bit1 = GPIO(GPIO.get_gpio_pin(2), 'out')
Floor_disp_bit2 = GPIO(GPIO.get_gpio_pin(3), 'out')


dir_disp = GPIO(GPIO.get_gpio_pin(1), 'out')
speed_disp = GPIO(GPIO.get_gpio_pin(0), 'out')
stop_disp = GPIO(GPIO.get_gpio_pin(7), 'out')



Floor_request_bit1 =     GPIO(GPIO.get_gpio_pin(4), 'in')
Floor_request_bit2 =     GPIO(GPIO.get_gpio_pin(5), 'in')
Floor_request_bit3 =     GPIO(GPIO.get_gpio_pin(6), 'in')


up_call_request = [0,0,0,0]
down_call_request = [0,0,0,0]


floor_pos = 0 
rate = 0

slow = 2
fast = 4



In [4]:
#led.write(1)

In [15]:
Floor_disp_bit1.write(1)
sleep(2)
Floor_disp_bit1.write(0)

In [16]:
Floor_disp_bit2.write(1)
sleep(2)
Floor_disp_bit2.write(0)

In [18]:
speed_disp.write(1) #Led on means going fast, Led off means going slow
sleep(2)
speed_disp.write(0)

In [19]:
stop_disp.write(1) #Led on means the elevator stop, off means its moving 
sleep(2)
stop_disp.write(0)

In [20]:
dir_disp.write(1) #Led on means going up off means down 
sleep(2)
dir_disp.write(0)

In [21]:
while(True):
    print(Floor_request_bit3.read(), Floor_request_bit2.read(), Floor_request_bit1.read())
    sleep(.5)

0 0 0
0 0 0
0 1 0
0 1 0
0 1 0
0 1 0
0 1 0
0 0 1
0 0 1
0 0 1
0 0 1
0 0 0
0 0 0
1 0 0
1 0 0
1 0 0
1 0 0
1 0 0
1 0 0
1 0 0
0 0 0
0 1 1
0 1 1
0 1 1
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
1 0 1
1 0 1
1 0 1
1 0 1
1 0 1
1 0 1
1 0 1
1 0 1
1 0 1
0 0 0
0 0 0
0 0 0


KeyboardInterrupt: 

In [22]:
while(True):
    bit = ( str(Floor_request_bit3.read()) +  str(Floor_request_bit2.read()) + str(Floor_request_bit1.read()) )
    if(bit == "001"):
        print("Floor 1")
    elif(bit == "010"):
        print("Floor 2")
    elif(bit == "011"):
        print("Floor 3")
    elif(bit == "100"):
        print("Floor 4")
    elif(bit == "101"):
        print("SoS")
        break
    sleep(1)

Floor 1
Floor 1
Floor 3
Floor 2
Floor 4
SoS


In [23]:
def set_reg(num):
    if (check_floor_pos() > num):
        down_call_request[num-1] = 1 
    elif (check_floor_pos() < num):
        up_call_request[num-1] = 1

In [24]:
def button_press():
    bit = ( str(Floor_request_bit3.read()) +  str(Floor_request_bit2.read()) + str(Floor_request_bit1.read()) )
    if(bit == "001"):
        set_reg(1)
        print("Floor 1")
    elif(bit == "010"):
        set_reg(2)
        print("Floor 2")
    elif(bit == "011"):
        set_reg(3)
        print("Floor 3")
    elif(bit == "100"):
        set_reg(4)
        print("Floor 4")
    elif(bit == "101"):
        print("SoS")
        set_reg(1)
        clear_floor_request(up_call_request,2)
        clear_floor_request(up_call_request,3)
        clear_floor_request(up_call_request,4)
        clear_floor_request(down_call_request,2)
        clear_floor_request(down_call_request,3)
        clear_floor_request(down_call_request,4)
    sleep(1)

In [25]:
def check_floor_pos():
    if(0<=floor_pos<10):
        Floor_disp_bit1.write(0)
        Floor_disp_bit2.write(0)
        return 1
    if(10<=floor_pos<20):
        Floor_disp_bit1.write(1)
        Floor_disp_bit2.write(0)
        return 2
    if(20<=floor_pos<30):
        Floor_disp_bit1.write(0)
        Floor_disp_bit2.write(1)
        return 3
    elif(30<=floor_pos<40):
        Floor_disp_bit1.write(1)
        Floor_disp_bit2.write(1)
        return 4
def safe_floor_pos():
    if(-2< floor_pos < 2 ):
        return True
    if(8 < floor_pos < 12):
        return True
    if(18 < floor_pos < 22 ):
        return True
    elif(28 < floor_pos < 32 ):
        return True

def next_floor_request(nex,call_request):
    return(call_request[nex-1] == 1)

def call_check(arr):
    return (1 in arr)

def clear_floor_request(arr,pos):
    arr[pos-1] = 0

def close_door():
    print("Door is close")
def open_door():
    print("Door is open")





In [11]:
print(call_check([0,0,1,0]))

True


In [26]:
def floor_stop(arr,pos):
    print("Stop on Floor: ", str(pos))
    floor_pos = (pos-1)*10
    
    stop_disp.write(1)
    sleep(1)
    open_door()
    sleep(4)
    close_door()
    sleep(1)
    rate = 0
    clear_floor_request(arr,pos)


In [25]:
up_call_request = [0,0,0,0]
down_call_request = [0,0,0,0]
floor_pos = 0
rate = 0
check_floor_pos()
dir_disp.write(0)

In [27]:
#State Machine
while(True):
#idle state
    dir_disp.write(0)
    button_press()


    while(call_check(up_call_request)): #up state
        
        stop_disp.write(0)
        dir_disp.write(1)
        button_press()
        
        
        print("Current Floor: "+ str(check_floor_pos()) + " Rate going up: " + str(rate)) 
       
        nex = check_floor_pos() + 1
    
    
        #speed control
        if(next_floor_request(nex,up_call_request)):
            speed_disp.write(0)
            rate = slow 
            sleep(1)
        elif not (next_floor_request(nex,up_call_request)):
            sleep(1)
            speed_disp.write(1)
            rate = fast    
            
        floor_pos = floor_pos + rate
        
        #check if there's a floor
        if((up_call_request[check_floor_pos()-1] == 1) and safe_floor_pos()):
            speed_disp.write(0)
            rate = 0 
            floor_stop(up_call_request, check_floor_pos())
            print("Idle on ", check_floor_pos())
    
    
    while(call_check(down_call_request)): #checks for floor calls in down state
        
        stop_disp.write(0)
        dir_disp.write(0)
        button_press()#checks if a button is press
        
        print("Current Floor: "+ str(check_floor_pos()) + " Rate going down: " + str(rate)) 
        nex = check_floor_pos() - 1
    
    
        #speed control
        if(next_floor_request(nex,down_call_request)):
            speed_disp.write(0)
            rate = -slow 
            sleep(1)
        elif not (next_floor_request(nex,down_call_request)):
            sleep(1)
            speed_disp.write(1)
            rate = -fast
        floor_pos = floor_pos + rate
        
        #check if there's a floor_request at the given floor
        if((down_call_request[check_floor_pos()-1] == 1) and safe_floor_pos()):
            speed_disp.write(0)
            rate = 0 
            floor_stop(down_call_request, check_floor_pos())
            print("Idle on ", check_floor_pos())
         

        
            

    
    
    


Floor 4
Current Floor: 1 Rate going up: 0
Current Floor: 1 Rate going up: 4
Current Floor: 1 Rate going up: 4
Current Floor: 2 Rate going up: 4
Floor 2
Current Floor: 2 Rate going up: 4
Current Floor: 3 Rate going up: 4
Current Floor: 3 Rate going up: 2
Current Floor: 3 Rate going up: 2
Current Floor: 3 Rate going up: 2
Current Floor: 3 Rate going up: 2
Stop on Floor:  4
Door is open
Door is close
Idle on  4
Floor 2
Current Floor: 4 Rate going down: 0
Current Floor: 3 Rate going down: -4
Current Floor: 3 Rate going down: -2
Current Floor: 3 Rate going down: -2
Current Floor: 3 Rate going down: -2
Current Floor: 2 Rate going down: -2
Current Floor: 2 Rate going down: -4
Stop on Floor:  2
Door is open
Door is close
Idle on  2
Floor 4
Current Floor: 2 Rate going up: 0
Current Floor: 2 Rate going up: 4
Current Floor: 2 Rate going up: 4
Current Floor: 3 Rate going up: 4
Current Floor: 3 Rate going up: 2
Current Floor: 3 Rate going up: 2
Floor 3
Current Floor: 3 Rate going up: 2
Stop on Floo

KeyboardInterrupt: 